In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps




# Import AnimalShelter Class from CRUDS Python module
from CRUDS import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# Define username and password for mongo client
username = "aacuser"
password = "aacuser"

#Initialize instance of AnimalShelter class
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.readAll({}))





#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dashboard')

#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())




app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Hr(),

    html.Div([
       dcc.Dropdown(
        id='filter_type',
        #Labels for the drop-down menu
        options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label': 'Disaster or Individual Tracking Rescue', 'value': 'Disaster'},
            {'label':'Reset','value':'Reset'}
        ],
        #Set initial value of filter to Reset. This will present as the standard table
        value='Reset'
        ),
        html.Div(id='dd-output-container')   
    ]),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native", #Native filter
        sort_action="native",#Native sort
        sort_mode="multi",
        column_selectable=False,
        row_selectable=True,
        row_deletable=False, #Make it so rows can't be deleted
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,#Specify page size (10 results)
    ),
    html.Br(),
     html.Hr(),
     html.H1('Devon Darling'), 
     html.Div(className='row',
         style={'display' : 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',

                ),
            html.Div(
                id='map-id',
                className='col s12 m6',
                )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter_type','value')]
             )
def update_dashboard(value):
# Value from the drop down menu is taken as a parameter and compared against the four types of filters
        if value == "Water":
            df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix",
"Chesapeake Bay Retriever",
"Newfoundland"]},"sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
        elif value == "Mountain":
            df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":{"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky",
"Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
        elif value == "Disaster":
            df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher",
"German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}))
        elif value == "Reset":
            df = pd.DataFrame.from_records(shelter.readAll({}))      
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')

        return (data,columns)

             
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
# Graph Callback
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # Convert data to dataframe
    dff = pd.DataFrame.from_dict(viewData)
    
    # Use value_counts to determine the number of times each breed is present in dataframe
    new_df = pd.DataFrame(dff.breed.value_counts().reset_index())
    # Set new dataframe column values
    new_df.columns = ['breed','value']
    # Return Pie Chart 
    return [
        dcc.Graph(
            figure = px.pie(new_df,values="value",names="breed")
        )
    ]
#Map Callback
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    #Setting values for latitude and longitude so it changes when pages change
    #Here I created variables to store lat and long
    lat = dff.loc[df.index[1], 'location_lat'] 
    long = dff.loc[df.index[1], 'location_long']
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat,long], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[lat,long], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]
    

app